In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *


# Initialize logger that adds to text file in current logging folder
from logging_utils import *
init_logger(save_dir=log_dir)
logger = get_logger()

In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=250
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
A=np.asmatrix(A)
AType='1D Laplacian'

retrain_freq=1
e1 = 1e-3
e2 = 1e-10
nmax_iter = 50
restart   = 64

sigma=0.02
DomainL=-5.0
DomainR=5.0
x=np.linspace(DomainL,DomainR,dim)



n_steps =500
InputDim=dim
HiddenDim=dim
HiddenDim2=dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 250 dim problem. Playing with SGD.")
#  The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-3,3]. ")



# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.sigma=sigma
        self.DomainL=DomainL
        self.DomainR=DomainR
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash





import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

2020-05-28 11:42:01 [INFO] <ipython-input-4-93f3238a1a0b> 2 - Benchmark 250 dim problem. Playing with SGD.


In [5]:
from src_dir import *

# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)



@timer
@nn_preconditioner_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,Err_list,ProbCount,restart,debug,refine,blist,reslist,IterErrList10,GmresRunTime):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)



@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [6]:
from src_dir import *




Err_list=[]
IterErrList10=[]
GmresRunTime=[]

NonML_Err_List=[]


debug=True

refine1=False
refine2=True

run_time=0.0
run_time_ML=0.0
trainTime_total=0.0
forwardTime_Total=0.0
run_time_ML_NoAdd=0.0

blist=[]
reslist=[]



for ProbIdx in range(n_steps):

    loc=4*np.sin(0.5*ProbIdx*np.abs(np.cos(0.5*ProbIdx)))
    b=Gauss_pdf(x,loc,sigma)
    b_norm=np.linalg.norm(b)



    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,Err_list,ProbIdx,restart,debug,refine1,blist,reslist,IterErrList10,GmresRunTime)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    Err_list=OutList1[1]
    IterErr0_AVG=OutList1[2]
    trainTime1=OutList1[3]
    forwardTime1=OutList1[4]
    blist=OutList1[5]
    reslist=OutList1[6]
    IterErrList10=OutList1[7]
    GmresRunTime=OutList1[8]

    OutList2,run_time2_ML=MLGMRES(A, b/b_norm, Out[-1], e2, nmax_iter,Err_list,ProbIdx,restart,debug,refine2,blist,reslist,IterErrList10,GmresRunTime)
    Out2=OutList2[0]
    Err_list=OutList2[1]
    IterErr0_AVG=OutList2[2]
    trainTime2=OutList2[3]
    forwardTime2=OutList2[4]
    blist=OutList2[5]
    reslist=OutList2[6]
    IterErrList10=OutList2[7]
    GmresRunTime=OutList2[8]

    forwardTime_Total=forwardTime1+forwardTime2+forwardTime_Total
    trainTime_total=trainTime2+trainTime1+trainTime_total
    run_time_ML=run_time1_ML+run_time2_ML+run_time_ML




    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Err=resid(A, NonML_Out1, b/b_norm)
    NonML_Err_List.append(NonML_Err[10])
    
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-1], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time=run_time1+run_time2+run_time
    
    print(ProbIdx)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)

logger.info("Runtime of forward function is:")
logger.info(forwardTime_Total)



0
1
2
loss: 2499442.0 0
loss: 2495440.75 1
loss: 2490003.5 2
loss: 2482526.5 3
loss: 2473072.5 4
loss: 2461861.75 5
loss: 2449035.0 6
loss: 2434907.25 7
loss: 2419519.5 8
loss: 2403250.0 9
loss: 2386291.75 10
loss: 2368909.75 11
loss: 2351219.0 12
loss: 2333374.75 13
loss: 2315447.25 14
loss: 2297436.5 15
loss: 2279407.0 16
loss: 2261378.5 17
loss: 2243367.5 18
loss: 2225385.0 19
loss: 2207440.75 20
loss: 2189546.5 21
loss: 2171714.5 22
loss: 2153954.0 23
loss: 2136272.5 24
loss: 2118662.0 25
loss: 2101142.5 26
loss: 2083718.0 27
loss: 2066392.5 28
loss: 2049167.875 29
loss: 2032033.625 30
loss: 2015008.75 31
loss: 1998094.0 32
loss: 1981290.375 33
loss: 1964599.25 34
loss: 1948021.875 35
loss: 1931559.25 36
loss: 1915212.0 37
loss: 1898980.875 38
loss: 1882866.5 39
loss: 1866869.0 40
loss: 1850989.125 41
loss: 1835226.5 42
loss: 1819581.375 43
loss: 1804054.0 44
loss: 1788644.25 45
loss: 1773351.75 46
loss: 1758176.625 47
loss: 1743118.5 48
loss: 1728177.0 49
loss: 1713352.125 50
loss

loss: 102109.1484375 421
loss: 101624.859375 422
loss: 101145.015625 423
loss: 100669.640625 424
loss: 100198.65625 425
loss: 99732.046875 426
loss: 99269.734375 427
loss: 98811.6953125 428
loss: 98357.921875 429
loss: 97908.34375 430
loss: 97462.890625 431
loss: 97021.5625 432
loss: 96584.3203125 433
loss: 96151.1015625 434
loss: 95721.890625 435
loss: 95296.6171875 436
loss: 94875.2890625 437
loss: 94457.828125 438
loss: 94044.234375 439
loss: 93634.421875 440
loss: 93228.3984375 441
loss: 92826.109375 442
loss: 92427.515625 443
loss: 92032.59375 444
loss: 91641.3125 445
loss: 91253.6015625 446
loss: 90869.453125 447
loss: 90488.84375 448
loss: 90111.71875 449
loss: 89738.0703125 450
loss: 89367.84375 451
loss: 89001.0 452
loss: 88637.53125 453
loss: 88277.359375 454
loss: 87920.5 455
loss: 87566.9375 456
loss: 87216.578125 457
loss: 86869.4375 458
loss: 86525.4375 459
loss: 86184.625 460
loss: 85846.890625 461
loss: 85512.2578125 462
loss: 85180.6796875 463
loss: 84852.1328125 464
l

loss: 48709.609375 855
loss: 48696.4921875 856
loss: 48683.4609375 857
loss: 48670.5234375 858
loss: 48657.69140625 859
loss: 48644.9453125 860
loss: 48632.28125 861
loss: 48619.71875 862
loss: 48607.22265625 863
loss: 48594.8203125 864
loss: 48582.52734375 865
loss: 48570.2890625 866
loss: 48558.15625 867
loss: 48546.1015625 868
loss: 48534.1328125 869
loss: 48522.25390625 870
loss: 48510.4375 871
loss: 48498.71484375 872
loss: 48487.07421875 873
loss: 48475.52734375 874
loss: 48464.046875 875
loss: 48452.64453125 876
loss: 48441.328125 877
loss: 48430.078125 878
loss: 48418.9140625 879
loss: 48407.8203125 880
loss: 48396.80859375 881
loss: 48385.8671875 882
loss: 48375.015625 883
loss: 48364.21875 884
loss: 48353.50390625 885
loss: 48342.859375 886
loss: 48332.29296875 887
loss: 48321.79296875 888
loss: 48311.375 889
loss: 48301.01953125 890
loss: 48290.734375 891
loss: 48280.5234375 892
loss: 48270.375 893
loss: 48260.3046875 894
loss: 48250.29296875 895
loss: 48240.3515625 896
loss

loss: 46649.97265625 1283
loss: 46648.0234375 1284
loss: 46646.08203125 1285
loss: 46644.1328125 1286
loss: 46642.1796875 1287
loss: 46640.25 1288
loss: 46638.3046875 1289
loss: 46636.3828125 1290
loss: 46634.4296875 1291
loss: 46632.5 1292
loss: 46630.5625 1293
loss: 46628.6328125 1294
loss: 46626.609375 1295
loss: 46610.93359375 1296
loss: 46599.0625 1297
loss: 46588.8828125 1298
loss: 46579.7265625 1299
loss: 46571.2578125 1300
loss: 46563.265625 1301
loss: 46555.671875 1302
loss: 46548.359375 1303
loss: 46541.2734375 1304
loss: 46534.40234375 1305
loss: 46527.671875 1306
loss: 46521.0859375 1307
loss: 46514.609375 1308
loss: 46508.234375 1309
loss: 46501.953125 1310
loss: 46495.7265625 1311
loss: 46489.578125 1312
loss: 46483.484375 1313
loss: 46477.4375 1314
loss: 46471.44140625 1315
loss: 46465.48046875 1316
loss: 46459.55078125 1317
loss: 46453.671875 1318
loss: 46447.8046875 1319
loss: 46441.96875 1320
loss: 46436.16015625 1321
loss: 46430.37890625 1322
loss: 46424.6171875 1323

loss: 44780.265625 1632
loss: 44774.97265625 1633
loss: 44769.6796875 1634
loss: 44764.375 1635
loss: 44759.0859375 1636
loss: 44753.7890625 1637
loss: 44748.484375 1638
loss: 44743.1875 1639
loss: 44737.875 1640
loss: 44732.578125 1641
loss: 44727.2734375 1642
loss: 44721.96484375 1643
loss: 44716.66796875 1644
loss: 44711.359375 1645
loss: 44706.05859375 1646
loss: 44700.75390625 1647
loss: 44695.4375 1648
loss: 44690.125 1649
loss: 44684.8203125 1650
loss: 44679.515625 1651
loss: 44674.19921875 1652
loss: 44668.8828125 1653
loss: 44663.57421875 1654
loss: 44658.24609375 1655
loss: 44652.9375 1656
loss: 44647.6171875 1657
loss: 44642.3046875 1658
loss: 44636.984375 1659
loss: 44631.65625 1660
loss: 44626.33203125 1661
loss: 44621.015625 1662
loss: 44615.6953125 1663
loss: 44610.37109375 1664
loss: 44605.046875 1665
loss: 44599.71484375 1666
loss: 44594.37890625 1667
loss: 44589.0703125 1668
loss: 44583.73828125 1669
loss: 44578.40234375 1670
loss: 44573.06640625 1671
loss: 44567.7382

loss: 41413.859375 2050
loss: 41405.4375 2051
loss: 41397.0234375 2052
loss: 41388.58984375 2053
loss: 41380.1640625 2054
loss: 41371.74609375 2055
loss: 41363.32421875 2056
loss: 41354.90625 2057
loss: 41346.46875 2058
loss: 41338.04296875 2059
loss: 41329.609375 2060
loss: 41321.1875 2061
loss: 41312.74609375 2062
loss: 41304.3203125 2063
loss: 41295.89453125 2064
loss: 41287.4609375 2065
loss: 41279.0390625 2066
loss: 41270.6015625 2067
loss: 41262.171875 2068
loss: 41253.7265625 2069
loss: 41245.2890625 2070
loss: 41236.859375 2071
loss: 41228.421875 2072
loss: 41219.9765625 2073
loss: 41211.5390625 2074
loss: 41203.1015625 2075
loss: 41194.65625 2076
loss: 41186.22265625 2077
loss: 41177.77734375 2078
loss: 41169.33203125 2079
loss: 41160.8828125 2080
loss: 41152.4453125 2081
loss: 41143.99609375 2082
loss: 41135.546875 2083
loss: 41127.109375 2084
loss: 41118.65234375 2085
loss: 41110.203125 2086
loss: 41101.7578125 2087
loss: 41093.3046875 2088
loss: 41084.8515625 2089
loss: 410

loss: 37774.3671875 2469
loss: 37765.3359375 2470
loss: 37756.30859375 2471
loss: 37747.265625 2472
loss: 37738.234375 2473
loss: 37729.1953125 2474
loss: 37720.15625 2475
loss: 37711.1171875 2476
loss: 37702.078125 2477
loss: 37693.03125 2478
loss: 37683.98828125 2479
loss: 37674.93359375 2480
loss: 37665.8828125 2481
loss: 37656.84375 2482
loss: 37647.78125 2483
loss: 37638.7109375 2484
loss: 37629.6640625 2485
loss: 37620.59375 2486
loss: 37611.5390625 2487
loss: 37602.484375 2488
loss: 37593.4140625 2489
loss: 37584.34765625 2490
loss: 37575.26171875 2491
loss: 37566.20703125 2492
loss: 37557.125 2493
loss: 37548.04296875 2494
loss: 37538.9609375 2495
loss: 37529.8828125 2496
loss: 37520.8046875 2497
loss: 37511.7265625 2498
loss: 37502.64453125 2499
Final loss: 37502.64453125 2500
Initial Training
3


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as pp
from src_dir import *

In [ ]:
AVG=np.zeros((500,1))
Err_Array=np.asarray(NonML_Err_List)
count=np.arange(0,500)
for j in range(0,500):
    AVG[j]=moving_average(np.asarray(Err_Array[:j]),j)

    
    
pp.plot(count,np.asarray(NonML_Err_List),'or',count[10:-1],AVG[10:-1],'oy')
pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('$n=600$ GMRES')
pp.savefig('GMRES.png')

In [ ]:
Err_Array_ML=np.asarray(IterErrList10)
AVGML=np.zeros((500,1))

for j in range(0,500):
    AVGML[j]=moving_average(np.asarray(Err_Array_ML[:j]),j)
pp.plot(count,np.asarray(Err_Array_ML),'ob',count,AVGML,'og')
pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('$n=600$ MLGMRES')
pp.savefig('MLGMRES.png')

In [ ]:
pp.plot(count,Err_Array_ML,'ob',label='MLGMRES error')
pp.plot(count,AVGML,'ok',label='Average MLGMRES error')
pp.plot(count,Err_Array,'or',label='GMRES error')
pp.plot(count[10:-1],AVG[10:-1],'oy',label='Average GMRES error')

pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('Error as a function of $b_i$ iteration ')
pp.legend(loc='best')
pp.savefig('Compare.png')